In [1]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os
#from dotenv import load_dotenv #

In [2]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [3]:
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
#import base64
import vertexai

from vertexai.generative_models import GenerativeModel, Part, FinishReason,GenerationConfig
import vertexai.preview.generative_models as generative_models

In [5]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [6]:
generation_config =GenerationConfig(
    temperature=0.8,
    top_p=0.8,
    max_output_tokens=8192,
)

In [7]:
model = GenerativeModel("gemini-1.5-flash-001")

In [8]:
df=pd.read_csv('info_solovenex_tweets.csv')

In [9]:
df.head()

,link,text,date,user,likes,quotes,retweets,comments
0,https://twitter.com/SOLOVENEX_/status/18367880...,‘𝗦𝗔𝗩𝗔’: ¡𝗘𝗟 𝗠𝗔́𝗦 𝗩𝗔𝗟𝗜𝗢𝗦𝗢! 🇻🇪 @QSavarino10 fue...,"Sep 19, 2024 · 3:22 PM UTC",@SOLOVENEX_,7,0,0,0
1,https://twitter.com/LAFC/status/18366166134682...,62' | GOAAALLL DAVID MARTÍNEZ #LAFCvATX 1-1,"Sep 19, 2024 · 4:01 AM UTC",@LAFC,184,1,15,11
2,https://twitter.com/SOLOVENEX_/status/18367318...,𝗗𝗔𝗩𝗜𝗗 𝗠𝗔𝗥𝗧𝗜́𝗡𝗘𝗭 anotó este miércoles su segund...,"Sep 19, 2024 · 11:39 AM UTC",@SOLOVENEX_,8,0,0,0
3,https://twitter.com/SOLOVENEX_/status/18364583...,🇻🇪 𝗝𝗨𝗔𝗡 𝗔𝗥𝗔𝗡𝗚𝗢 𝗝𝗥 (𝟮𝟬𝟬𝟲) escribió su nombre en...,"Sep 18, 2024 · 5:32 PM UTC",@SOLOVENEX_,16,0,4,0
4,https://twitter.com/SOLOVENEX_/status/18361468...,¡Ya son 𝟭𝟬𝟬 batallas con el Parma! 🇻🇪 El bari...,"Sep 17, 2024 · 8:54 PM UTC",@SOLOVENEX_,1026,5,139,5


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      1134 non-null   object
 1   text      1112 non-null   object
 2   date      1134 non-null   object
 3   user      1134 non-null   object
 4   likes     1134 non-null   int64 
 5   quotes    1134 non-null   int64 
 6   retweets  1134 non-null   int64 
 7   comments  1134 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 71.0+ KB


In [11]:
df['text'].head()

0    ‘𝗦𝗔𝗩𝗔’: ¡𝗘𝗟 𝗠𝗔́𝗦 𝗩𝗔𝗟𝗜𝗢𝗦𝗢! 🇻🇪  @QSavarino10 fue...
1         62' | GOAAALLL DAVID MARTÍNEZ  #LAFCvATX 1-1
2    𝗗𝗔𝗩𝗜𝗗 𝗠𝗔𝗥𝗧𝗜́𝗡𝗘𝗭 anotó este miércoles su segund...
3    🇻🇪 𝗝𝗨𝗔𝗡 𝗔𝗥𝗔𝗡𝗚𝗢 𝗝𝗥 (𝟮𝟬𝟬𝟲) escribió su nombre en...
4    ¡Ya son 𝟭𝟬𝟬 batallas con el Parma! 🇻🇪  El bari...
Name: text, dtype: object

In [12]:
##poniendolo como df es como mejor contesta....

contexto = f"""sos un agente de social media y neceito que analices estos Tweets : {df}"""


pregunta = f""" Necesito que a partir de esa informacion realices una lista de los jugadores de los cuales se habla, de que nacionalidad son y que descendencia tienen """
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses:
    print(response.text, end="")


¡Entendido! Puedo ayudarte a identificar los jugadores, nacionalidades y descendencias de los Tweets que proporcionaste. Sin embargo, la información proporcionada no es suficiente para determinar la descendencia de los jugadores. La descendencia es un tema complejo y personal que no se puede inferir solo por su nacionalidad o información deportiva.

Puedo ayudarte a identificar los jugadores y sus nacionalidades:

**Jugadores mencionados en los Tweets:**

* **Q. Savarino:** No se especifica la nacionalidad, pero por el contexto del Tweet, es probable que sea venezolano.
* **David Martínez:** No se especifica la nacionalidad, pero por el contexto del Tweet, es probable que sea venezolano.
* **Juan Arango Jr.:** No se especifica la nacionalidad, pero por el contexto del Tweet, es probable que sea venezolano.
* **Leandro Rodríguez:** Se especifica que es venezolano.
* **Kervin Andrade:** Se especifica que es venezolano.
* **Kelsy Kevin:** No se especifica la nacionalidad, pero por el cont

In [13]:
##poniendolo como df es como mejor contesta....

contexto = f"""sos un agente de social media y neceito que analices estos Tweets : {df}"""


pregunta = f""" Necesito que a partir de esa informacion realices una lista de los jugadores de los cuales se habla, de que nacionalidad son y que descendencia tienen.
Muchos de los jugadores nombrados en esta cuenta de tweeter son jugadores que tienen padre y/o madre venezolanos pero nacieron en otro pais. Lo que me interesa es saber si
los jugadores tienen alguno de sus padres venezolanos o ambos, y donde nacieron ellos, si en Venezuela o en otro pais. La informacion esta en: {df} """
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses:
    print(response.text, end="")

¡Entendido! Puedo ayudarte a analizar la información de los tweets para identificar la nacionalidad y descendencia de los jugadores mencionados. Sin embargo, necesito acceso a la información completa de los tweets para poder procesarla.  

Por favor, compárteme la información de los tweets, incluyendo el texto, la fecha y el nombre de usuario.  Puedo ayudarte a identificar:

* **Nombre del jugador:** Extraeré los nombres de los jugadores mencionados en los tweets.
* **Nacionalidad:**  Determinar la nacionalidad del jugador, si se menciona en el tweet.
* **Descendencia Venezolana:**  Identificar si el jugador tiene uno o ambos padres venezolanos, basándome en la información del tweet.
* **Lugar de nacimiento:**  Identificar si el jugador nació en Venezuela o en otro país.

Una vez que tenga la información completa, puedo crear una lista con los detalles de cada jugador. 

¡Espero tus datos para poder ayudarte! 


In [15]:

contexto = f"""sos un agente de social media y neceito que analices estos Tweets : {df}"""


pregunta = f""" Necesito que a partir de esa informacion realices una lista de los jugadores de los cuales se habla, de que nacionalidad son, es decir en donde nacieron, y que descendencia tienen
, es decir de que pais son sus padres.
Muchos de los jugadores nombrados en esta cuenta de tweeter son jugadores que tienen padre y/o madre venezolanos pero nacieron en otro pais. Lo que me interesa es saber si
los jugadores tienen alguno de sus padres venezolanos o ambos, y donde nacieron ellos, si en Venezuela o en otro pais. La informacion esta en: {df} """
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses3 = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses3:
    print(response.text, end="")

¡Entendido! Puedo ayudarte a analizar la información de los jugadores venezolanos en la cuenta de Twitter @SOLOVENEX_. 

Para procesar la información de manera eficiente, necesito que me proporciones el archivo con los datos de los Tweets. Puedes enviarme el archivo en formato CSV o similar. 

Una vez que tenga los datos, puedo crear una lista con la siguiente información para cada jugador:

* **Nombre del Jugador:** 
* **Nacionalidad:** (País de nacimiento)
* **Descendencia:** (País de origen de sus padres)
* **Lugar de Nacimiento:** (Venezuela u otro país)

Si necesitas información adicional, como el equipo actual del jugador, o algún otro dato relevante, por favor indícalo.

Estoy listo para ayudarte a analizar los datos y generar la lista de jugadores venezolanos. ¡Espero tus datos!


Aca por lo menos puede leer el DF y trae mas nombres


In [16]:

contexto = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {df}"""


pregunta = f"""Necesito que a partir del contenido de los tweets realices una lista o tabla de todos los jugadores de los cuales se habla, de que nacionalidad son, es decir en donde nacieron, y que descendencia
tienen, es decir en que pais nacieron sus padres.
Muchos de los jugadores nombrados en esta cuenta de tweeter son jugadores que tienen padre y/o madre venezolanos pero nacieron en otro pais. Lo que me interesa es saber si
los jugadores tienen alguno de sus padres venezolanos o ambos, y donde nacieron ellos, si en Venezuela o en otro pais. La informacion esta en: {df} """
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses:
    print(response.text, end="")

## Jugadores Venezolanos: Nacionalidad y Desendencia

A continuación, se presenta una lista de los jugadores mencionados en los tweets, junto con su nacionalidad y la información disponible sobre la nacionalidad de sus padres:

| Jugador | Nacionalidad | Padre Venezolano | Madre Venezolana | Lugar de Nacimiento de los Padres |
|---|---|---|---|---|
| **Q. Savarino** | Venezolano | Sí | Sí | Venezuela |
| **David Martínez** | Venezolano | Sí | Sí | Venezuela |
| **Juan Arango Jr.** | Venezolano | Sí | Sí | Venezuela |
| **Leandro Rodríguez** | Venezolano | Sí | Sí | Venezuela |
| **Kervin Andrade** | Venezolano | Sí | Sí | Venezuela |
| **Kelsy Kevin** | Venezolano | Sí | Sí | Venezuela |
| **Eliel Sánchez** | Venezolano | Sí | Sí | Venezuela |
| **Wuilker Faríñez** | Venezolano | Sí | Sí | Venezuela |
| **José Manuel Velázquez** | Venezolano | Sí | Sí | Venezuela |
| **Jefferson Savarino** | Venezolano | Sí | Sí | Venezuela |
| **Yangel Herrera** | Venezolano | Sí | Sí | Venezuela |
| 

In [26]:

contexto = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {df}"""


pregunta = f"""Necesito que a partir del contenido de los tweets realices una lista o tabla de todos los jugadores de los cuales se habla, de que nacionalidad son, es decir en donde nacieron, y que descendencia
tienen, es decir en que pais nacieron sus padres.
Muchos de los jugadores nombrados en esta cuenta de tweeter son jugadores que tienen padre y/o madre venezolanos pero nacieron en otro pais. Lo que me interesa es saber si
los jugadores tienen alguno de sus padres venezolanos o ambos, y donde nacieron ellos, si en Venezuela o en otro pais. La informacion esta en: {df} 
Si no sabes la informacion responde que no sabes. """
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses2 = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses2:
    print(response.text, end="")

## Análisis de Jugadores Venezolanos en Tweets

He analizado los tweets proporcionados y he recopilado la información sobre los jugadores mencionados, su nacionalidad, lugar de nacimiento y la nacionalidad de sus padres:

| Jugador | Nacionalidad | Lugar de Nacimiento | Nacionalidad Padre | Nacionalidad Madre |
|---|---|---|---|---|
| **Q. Savarino** | Venezolano | Venezuela | Venezolano | Venezolana |
| **David Martínez** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Juan Arango Jr.** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Yeferson Soteldo** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Leandro Rodríguez** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Kervin Andrade** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Kelsy Kevin** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Eliel Sánchez** | Venezolano | Venezuela | Venezolano | Venezolana |
| **Wuilker Faríñez** | Venezolano | Venezuela | Venezolano | Venezolana |

In [29]:
contexto = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {df}"""


pregunta = f"""Necesito que a partir del contenido de los tweets realices una lista o tabla de todos los jugadores de los cuales se habla, de que nacionalidad son, es decir en donde nacieron, y que descendencia
tienen, es decir en que pais nacieron sus padres.
Muchos de los jugadores nombrados en esta cuenta de tweeter son jugadores que tienen padre y/o madre venezolanos pero nacieron en otro pais. Lo que me interesa es saber si
los jugadores tienen alguno de sus padres venezolanos o ambos, y donde nacieron ellos, si en Venezuela o en otro pais. La informacion esta en: {df} 
Si no sabes la informacion responde que no sabes.
Y si encontras la informacion que te solicito en los tweets quiero que lo cites y me digas de donde la obtuviste """
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses2 = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses2:
    print(response.text, end="")

¡Claro que sí! Puedo ayudarte a analizar los tweets y crear una tabla con la información que necesitas. Sin embargo, me he encontrado con un pequeño problema: la información que buscas no se encuentra en los tweets que me has proporcionado. La cuenta de Twitter @SOLOVENEX_ habla de muchos jugadores venezolanos, pero no menciona la nacionalidad de sus padres ni el lugar de nacimiento de estos. 

Para obtener esa información, necesitaría buscar en otras fuentes como artículos de prensa, perfiles de jugadores o sitios web de clubes. 

¿Te gustaría que buscara en otras fuentes para intentar encontrar la información que necesitas? 


In [ ]:
import pandas as pd

df = pd.read_csv('tu_archivo.csv', header=None)  # Si no tienes encabezados
cadena_unica = ' '.join(df.values.flatten())
print(cadena_unica)

In [ ]:
contexto = f"""Sos un agente de social media y necesito que analices estos Tweets con detalle : {cadena_unica}"""


pregunta = f"""Necesito que a partir del contenido de los tweets realices una lista o tabla de todos los jugadores de los cuales se habla, de que nacionalidad son, es decir en donde nacieron, y que descendencia
tienen, es decir en que pais nacieron sus padres.
Muchos de los jugadores nombrados en esta cuenta de tweeter son jugadores que tienen padre y/o madre venezolanos pero nacieron en otro pais. Lo que me interesa es saber si
los jugadores tienen alguno de sus padres venezolanos o ambos, y donde nacieron ellos, si en Venezuela o en otro pais. La informacion esta en: {cadena_unica} 
Si no sabes la informacion responde que no sabes.
Y si encontras la informacion que te solicito en los tweets quiero que lo cites y me digas la referencia de donde la obtuviste"""
# Prompt con el contexto y la pregunta
prompt = f"{contexto}\n\n{pregunta}"

# Generar la respuesta utilizando el prompt actualizado
responses2 = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
  # Genera la respuesta
for response in responses2:
    print(response.text, end="")